# 🚚 Logística y Transporte: costo por entrega, SLA, ocupación
KPIs: costo/km, OTIF, ocupación, distancia vacía, tiempo de ciclo.

## Casos de uso
- Ruteo óptimo con ventanas de tiempo
- Predicción de ETA y retrasos
- Optimización de ocupación y backhauling
- Control de temperatura y seguridad de carga

In [ ]:
# Estimación simple de ETA
def eta(dist_km, vel_kmh=50):
    return round(dist_km / max(vel_kmh,1), 2)
eta(120)

## Arquitectura
- Telemetría IoT y mapas, enriquecimiento geoespacial
- Optimización y simulación de rutas
- APIs para clientes y tableros de flota

## 💼 Valor en Logística
- OTIF ↑ y costo por entrega ↓ con ruteo óptimo.
- Menos tiempos muertos y backhauling inteligente.
- Trazabilidad y seguridad de carga.


In [ ]:
# Práctica: ETA y OTIF (toy)
import pandas as pd
entregas = pd.DataFrame({
    'dist_km':[10, 25, 60, 5],
    'ventana_horas':[0.5, 1.0, 1.5, 0.3],
    'vel_kmh':[40, 50, 45, 30],
})
entregas['ETA_h'] = entregas['dist_km']/entregas['vel_kmh']
entregas['OTIF'] = (entregas['ETA_h'] <= entregas['ventana_horas']).astype(int)
print(entregas)
print('OTIF %:', round(100*entregas['OTIF'].mean(),1))


## 🏗️ Arquitectura de telemetría y ruteo
- Telemetría IoT + mapas; enriquecimiento geoespacial.
- Optimizadores de rutas y simulación; tableros de flota.
- APIs de seguimiento y notificaciones a cliente.
